In [ ]:
!ls

In [ ]:
%cd drive/My\ Drive/2048-api-master

/content/drive/My Drive/2048-api-master


提取训练数据，DATA.csv

In [ ]:
import csv
import numpy as np
import time

datas = []
labels = []

def step2array(step):
    vec = np.zeros(4, dtype = bool)
    vec[step] = 1
    return vec

start = time.time()
with open("DATA.csv", "r") as f:
    csv_read = csv.reader(f)
    for line in csv_read:
        board = np.zeros((4, 4))
        for i in range(16):
            board[i // 4][i % 4] = int(line[i]) / 11.0
        boardT = board.T
        step = int(line[16])
        datas.append(np.hstack((board, boardT)))
        labels.append(step2array(step))
print("time: ", time.time() - start)

datas = np.array(datas)
labels = np.array(labels)

print(datas.shape)
print(labels.shape)

time:  165.07440567016602
(8178972, 4, 8)
(8178972, 4)


定义模型RCNN

In [ ]:
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Reshape
from tensorflow.keras.layers import Flatten, Dropout, Conv2D, TimeDistributed
from tensorflow.keras.layers import BatchNormalization, Activation, concatenate


def RCNN_model():
    inputs = Input((4, 8))

    lstm1 = LSTM(256, return_sequences = True, activation = 'relu')(inputs)
    lstm2 = LSTM(256, return_sequences = True, activation = 'relu')(lstm1)
    lstm3 = LSTM(256, return_sequences = True, activation = 'relu')(lstm2)
    lstm4 = LSTM(256, return_sequences = False, activation = 'relu')(lstm3)

    flatten1 = Flatten()(lstm4)
    de1 = Dense(256, activation = 'relu')(flatten1)

    reshape1 = Reshape((4, 8, 1))(inputs)
    conv1 = Conv2D(filters = 1, kernel_size = 2, strides = 2)(reshape1)
    
    flatten2 = Flatten()(conv1)
    de2 = Dense(256, activation = 'relu')(flatten2)
    # reshape2 = Reshape((8, 256))(de2)

    hidden = concatenate([de1, de2])
    flatten3 = Flatten()(hidden)

    outputs = Dense(4, activation = 'softmax')(flatten3)
    model = Model(inputs, outputs)
    return model

训练模型

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import os

model = RCNN_model()
filepath = "checkpoints/checkpoint.hdf5"

model.summary()

model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics=['accuracy'])

checkpoint = ModelCheckpoint(filepath, monitor="val_acc", save_best_only = False, verbose=1, mode="auto")

if os.path.exists(filepath):
    model.load_weights(filepath)
    print("checkpoint_loaded")

model.fit(datas, labels, batch_size = 1000, epochs = 10, callbacks = [checkpoint])

测试模型

In [ ]:
from game2048.game import Game
from game2048.displays import Display, IPythonDisplay
from game2048.agents import Agent, RandomAgent, ExpectiMaxAgent
import numpy as np
from models.refer_model import refer_model

model = RCNN_model()
model.load_weights("checkpoints/checkpoint.hdf5")

def reshape_board(board):
    res = np.zeros((4, 4), dtype = float)
    for i in range(4):
        for j in range(4):
            k = int(board[i, j])
            if k != 0:
                res[i, j] = np.log2(k) / 11

    res1 = res.T
    return np.hstack((res, res1))

game3 = Game(score_to_win = 2048, random = False)
display3 = Display()

while game3.end == 0:
    display3.display(game3)
    # agent1 = ExpectiMaxAgent(game3)
    board = np.array([reshape_board(game3.board)])
    prediction = model.predict(board)
    step = np.argmax(prediction, axis = 1)
    # step = agent1.step()

    game3.move(step)

生成评价log

In [ ]:
!python evaluate.py >> EE369_evaluation.log

2020-06-17 11:11:01.668430: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-06-17 11:11:03.548965: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-17 11:11:03.553071: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-06-17 11:11:03.553126: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (792d4b7f141f): /proc/driver/nvidia/version does not exist
2020-06-17 11:11:03.553539: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX512F
2020-06-17 11:11:03.561663: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2000175000 Hz
2020-06-17 11:11:03.562020: I tensorflow/compiler/xla/servi

生成agent指纹

In [ ]:
!python generate_fingerprint.py